In [ ]:
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import adi
import zmq
from pynq import Overlay
from pynq import GPIO

In [ ]:
ol = Overlay("/home/xilinx/jupyter_notebooks/fm_radio/fm_radio.bit")
ol.download(dtbo="/home/xilinx/jupyter_notebooks/base/pl.dtbo")
!systemctl restart iiod

In [ ]:
reset = GPIO(GPIO.get_gpio_pin(32), 'out')
reset.write(1)

In [ ]:
fc = 89.7e6 # Hz
fs = 1.92e6 # Hz

sdr = adi.ad9361(uri="local:")
sdr.sample_rate = int(fs)

# Config Tx
sdr.tx_rf_bandwidth = int(fs)
sdr.tx_lo = int(fc)
sdr.rx_hardwaregain_chan0 = -90
sdr.tx_enabled_channels = [0]
sdr.tx_cyclix_buffer = True

# Create Tx waveform
N_tx = 10000
t = np.arange(N_tx) / fs
f = 100e3 # Hz
s = (2**13)*np.exp(1j*2*np.pi*f*t)

# Config Rx
N_rx = 10000
sdr.rx_lo = int(fc)
sdr.rx_rf_bandwidth = int(fs)
sdr.rx_buffer_size = N_rx
sdr.gain_control_mode_chan0 = 'slow_attack'
sdr.rx_enabled_channels = [0]
# sdr.rx_hardwaregain_chan0 = -10.0 # dB, increase to increase the receive gain, but be careful not to saturate the ADC

In [ ]:
sdr.tx(s)

In [ ]:
reset.write(0)

In [ ]:
def zmq_pub(ip, port):
    context = zmq.Context()
    socket = context.socket(zmq.PUB)
    socket.bind(f"tcp://{ip}:{port}")
    return socket

In [ ]:
# Clear buffer just to be safe
for i in range (0, 10):
    raw_data = sdr.rx()

In [ ]:
socket = zmq_pub(ip='192.168.2.11', port=5050)

for i in range(10):
    r = sdr.rx()
    audio = np.real(r) + (np.imag(r) << 16)
    socket.send(audio.astype(np.float32))

In [ ]:
socket.close()